# Extract Data from Snapdeal
### Product : Name, Actual price, Discount price, Rating, Image url, Product url
### Link for mobile catregory :

In [1]:
# importing libraries
from bs4 import BeautifulSoup

# Firefox & Chorme
from selenium import webdriver

import pandas as pd
import time
import csv
import re

In [2]:
# Startup the webdriver
driver = webdriver.Chrome('C:/webdriver/chromedriver.exe')

## Product 0 - 20

In [7]:
# Extracting the link of the 1 products
url_page_1 = 'https://www.snapdeal.com/products/mobiles-mobile-phones?sort=plrty'

driver.get(url_page_1)
soup = BeautifulSoup(driver.page_source, 'html.parser')

results = soup.find_all('div',{'class':'product-tuple-listing'})

item = results[0]
# URL of a product
atag = item.find('a',{'class':'dp-widget-link noUdLine hashAdded'})
url = atag.get('href')
url

'https://www.snapdeal.com/product/redmi-9-64gb-4-gb/646372370923#bcrumbLabelId:175'

In [9]:
# Extracting the link of the 0-20 products

def extract_record(item):
    """Extract and return url from a single record"""
    # URL of a product
    atag = item.find('a',{'class':'dp-widget-link noUdLine hashAdded'})
    url = atag.get('href')

    return url

# Storing the 0-20 product url
product_20 = []
results = soup.find_all('div',{'class':'product-tuple-listing'})
# iterate to get 1-20 product link
for item in results:
    record = extract_record(item)
    if record:
        product_20.append(record)

## Product 21 - 40

In [13]:
# Extracting the link of the 21-40 products
url_page_2 = 'https://www.snapdeal.com/acors/json/product/get/search/175/20/20?q=&sort=plrty&brandPageUrl=&keyword=&searchState=k3=true|k5=0|k6=0|k7=7UuJ4EMACA==|k8=0&pincode=&vc=&webpageName=categoryPage&campaignId=&brandName=&isMC=false&clickSrc=unknown&showAds=true&cartId=&page=cp'

driver.get(url_page_2)
soup = BeautifulSoup(driver.page_source, 'html.parser')

results = soup.find_all('div', {'class': 'product-tuple-listing'})
item = results[0]
# getting url
atag = item.find('a', {'class': 'dp-widget-link noUdLine'})
url = atag.get('href')


def extract_record(item):
    """Extract and return url from a single record"""
    # URL of a product
    atag = item.find('a', {'class': 'dp-widget-link noUdLine'})
    url = atag.get('href')
    return url


# Storing the 21-40 product url
product_40 = []
results = soup.find_all('div', {'class': 'product-tuple-listing'})

# iterate to get 21-40 product link
for item in results:
    record = extract_record(item)
    if record:
        product_40.append(record)

## Product 41 - 60

In [24]:
# Extracting the link of the 41-60 products
url_page_3 = 'https://www.snapdeal.com/acors/json/product/get/search/175/40/20?q=&sort=plrty&brandPageUrl=&keyword=&searchState=k3=true|k5=0|k6=0|k7=///f//8ACA==|k8=0&pincode=&vc=&webpageName=categoryPage&campaignId=&brandName=&isMC=false&clickSrc=unknown&showAds=true&cartId=&page=cp'
driver.get(url_page_3)
soup = BeautifulSoup(driver.page_source,'html.parser')

results = soup.find_all('div',{'class':'product-tuple-listing'})
item = results[0]
atag = item.find('a',{'class':'dp-widget-link noUdLine'})
# getting url
url = atag.get('href')

def extract_record(item):
    """Extract and return url from a single record"""
    # URL of a product
    atag = item.find('a',{'class':'dp-widget-link noUdLine'})
    url = atag.get('href')
    return url

# Storing the 41-60 product url
product_60 = []
results = soup.find_all('div',{'class':'product-tuple-listing'})

# iterate to get 41-60 product link
for item in results:
    record = extract_record(item)
    if record:
        product_60.append(record)

In [33]:
# Collect all product url in one list
product_links = product_20+product_40+product_60

In [37]:
def get_product(url):
    """Extract and return product details from a single record"""

    driver.get(f'{url}')
    result = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Product Name
    try:
        product_name = result.find('h1', {'itemprop': 'name'}).text.strip()
    except AttributeError:
        product_name = 'NAN'
    
    # Actual price   
    try:
        actPrice = result.find('div',{'class' : 'pdpCutPrice'}).get_text()
    except AttributeError:
        actPrice = 'NAN'
    
    # Discount price
    try:
        disPrice = result.find('span',{'class':'payBlkBig'}).text
    except AttributeError:
        disPrice = 'NAN'
              
    # Product Rating
    try:
        rating_count = result.find('span', {'class': 'avrg-rating'}).text.strip()
    except AttributeError:
        rating_count = '0'

    # Image URL
    try:
        imgtag = result.find('img',{'class':'cloudzoom'})
        img_url = imgtag.get('src')
    except AttributeError:
        img_url = 'NAN'
    
    Productdetails = {
            'Name': product_name,
            'SalesPrice': disPrice,
            'OriginalPrice': actPrice,
            'Rating': rating_count,
            'ProductLink': url,
            'ImageLink': img_url
        }

    return Productdetails

# Store the values
productList = []

# Iterating the URL List
for item in product_links:
    records = get_product(item)
    
    if records:
        productList.append(records)
        
        # sleep for 1sec
        time.sleep(1)

In [38]:
print(f'There are total {len(productList)} products')

There are total 55 products


## Save data in csv file

In [60]:
dataset = pd.DataFrame(productList)
dataset.to_csv('Productdata.csv', index=False)

# Data Cleaning

In [61]:
df = pd.read_csv('Productdata.csv')

In [64]:
df['OriginalPrice'] = df['OriginalPrice'].str.replace(r"[^0-9]",'')

<ipython-input-64-f6fa2c63a0d7>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['OriginalPrice'] = df['OriginalPrice'].str.replace(r"[^0-9]",'')


In [66]:
df['SalesPrice'] = df['SalesPrice'].str.replace(r"[^0-9]",'')

<ipython-input-66-701fe8609850>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['SalesPrice'] = df['SalesPrice'].str.replace(r"[^0-9]",'')


In [67]:
df['Rating'] = df['Rating'].str.replace(r"[()]",'')

<ipython-input-67-5338e2eec5f4>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Rating'] = df['Rating'].str.replace(r"[()]",'')


# Snapdeal Product Dataset

In [69]:
# Save clean data as csv
df.to_csv('SnapDealProducts.csv',index=0)

In [70]:
import pandas as pd
new_df = pd.read_csv('SnapDealProducts.csv')
new_df

,Name,SalesPrice,OriginalPrice,Rating,ProductLink,ImageLink
0,"Redmi 9 ( 64GB , 4 GB ) Black",9499,10999.0,3.6,https://www.snapdeal.com/product/redmi-9-64gb-...,https://n2.sdlcdn.com/imgs/j/r/t/Redmi-9-64GB-...
1,"Xolo Era 2 ( 8GB , 1 GB ) Black",4463,NaN,3.8,https://www.snapdeal.com/product/xolo-era-2-8g...,https://n3.sdlcdn.com/imgs/h/h/d/1-31100.jpg
2,"Gionee s11 ( 32GB , 4 GB ) Black",7000,15399.0,0.0,https://www.snapdeal.com/product/gionee-s11-32...,https://n3.sdlcdn.com/imgs/j/s/0/Gionee-s11-32...
3,"Realme Narzo 20 ( 64GB , 4 GB ) Victory Blue",11999,12999.0,4.0,https://www.snapdeal.com/product/realme-narzo-...,https://n1.sdlcdn.com/imgs/j/r/o/Realme-Narzo-...
4,"Gionee s11 lite ( 32 GB , 4 GB ) Blue",7004,8999.0,3.4,https://www.snapdeal.com/product/gionee-blue-s...,https://n3.sdlcdn.com/imgs/i/f/r/Gionee-Blue-s...
5,"Xolo Era 2 ( 8GB , 1 GB ) Pyramid Gold",4463,NaN,3.8,https://www.snapdeal.com/product/xolo-era-2-8g...,https://n4.sdlcdn.com/imgs/h/h/d/1-1bfa8.jpg
6,"Realme C11 ( 32GB , 2 GB ) Grey",8999,9999.0,4.6,https://www.snapdeal.com/product/realme-c11-32...,https://n1.sdlcdn.com/imgs/j/f/p/Realme-C11-32...
7,"Redmi Note 10 ( 64GB , 4 GB ) Aqua Green",13999,15999.0,0.0,https://www.snapdeal.com/product/redmi-note-10...,https://n1.sdlcdn.com/imgs/j/s/m/Redmi-Note-10...
8,"Realme Narzo 10A ( 32GB , 3 GB ) Blue",9803,10999.0,0.0,https://www.snapdeal.com/product/realme-narzo-...,https://n1.sdlcdn.com/imgs/j/f/c/Realme-Narzo-...
9,"Blackberry KEYONE ( 32 MB , 4 GB ) Black & Grey",23999,31999.0,3.0,https://www.snapdeal.com/product/blackberry-bl...,https://n4.sdlcdn.com/imgs/g/t/d/Blackberry-Bl...
